In [1]:
from numpy.random import seed
seed(0)
from tensorflow import set_random_seed
set_random_seed(0)

import numpy as np
from keras.layers import Input, Dense
from keras.models import Model
from keras import backend as K

Using TensorFlow backend.


In [2]:
# helper function loss function
def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true), axis=-1)) 
    
embedding_dim = 1 # dimension of embedding    

In [3]:
X_train =  np.load("J2_Xtr.npy")
X_val =  np.load("J2_Xval.npy")
X_test =  np.load("J2_Xtest.npy")

X_train_noisy = X_train + np.random.normal(loc=0.0, scale=1.0, size=X_train.shape)
X_val_noisy = X_val + np.random.normal(loc=0.0, scale=1.0, size=X_val.shape)
X_test_noisy = X_test + np.random.normal(loc=0.0, scale=1.0, size=X_test.shape)

# binary label

In [4]:
# load labels
y_train_bin =  np.load("J2_ytrBin.npy")
y_val_bin =  np.load("J2_yvalBin.npy")
y_test_bin =  np.load("J2_ytestBin.npy")

In [5]:
# normal autoencoder (Selu)

In [6]:
# joint (multi task learning model) loss

from numpy.random import seed
seed(0)
from tensorflow import set_random_seed
set_random_seed(0)

x_dim = X_train.shape[1]
x_input = Input(shape=(x_dim,))


hidden_enc = Dense(2, activation='selu')(x_input)
# embedding is the user-item embedding (encoded representation)
embedding = Dense(embedding_dim, activation='selu')(hidden_enc)
hidden_dec = Dense(2, activation='selu')(embedding)
# reconstructed_input is the (decoded) reconstruction of the input 
# (lossy reconstruction)
reconstructed_input = Dense(x_dim, activation='selu')(hidden_dec)
predictor = Dense(1, activation='selu')(hidden_dec)



# Model(input, output): map the input to its reconstruction
# definition of autoencoder
joint_model = Model(inputs=x_input, outputs=[reconstructed_input, predictor])


joint_model.summary()

joint_model.compile(optimizer='rmsprop', loss=root_mean_squared_error)

joint_model.fit(
    x = [X_train_noisy], 
    y = [X_train, y_train_bin],
    epochs = 5,
    batch_size = 32,
    validation_data = ([X_val_noisy], [X_val, y_val_bin])
)
joint_model.evaluate([X_test_noisy], [X_test, y_test_bin])

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 2)            0                                            
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 2)            6           input_1[0][0]                    
__________________________________________________________________________________________________
dense_2 (Dense)                 (None, 1)            3           dense_1[0][0]                    
__________________________________________________________________________________________________
dense_3 (Dense)                 (None, 2)            4           dense_2[0][0]                    
__________________________________________________________________________________________________
dense_4 (D

[nan, nan, nan]

In [7]:
# 2 single (consecutive) loss

# separate
from numpy.random import seed
seed(0)
from tensorflow import set_random_seed
set_random_seed(0)



x_dim = X_train.shape[1]
x_input = Input(shape=(x_dim,))


hidden_enc = Dense(2, activation='selu')(x_input)
# embedding is the user-item embedding (encoded representation)
embedding = Dense(embedding_dim, activation='selu')(hidden_enc)
hidden_dec = Dense(2, activation='selu')(embedding)
# reconstructed_input is the (decoded) reconstruction of the input 
# (lossy reconstruction)
reconstructed_input = Dense(x_dim, activation='selu')(hidden_dec)


# Model(input, output): map the input to its reconstruction
# definition of autoencoder
autoencoder = Model(x_input, reconstructed_input)


# map the input to its embedding (encoded representation)
encoder = Model(x_input, embedding)


autoencoder.summary()

autoencoder.compile(optimizer='rmsprop', loss=root_mean_squared_error)

autoencoder.fit(
    X_train_noisy, X_train,
    epochs=5,
    batch_size=32,
    validation_data = (X_val_noisy, X_val)
)

# part 2
predictor = Dense(1, activation='selu')(embedding)


model = Model(x_input, predictor)

model.compile(optimizer='rmsprop', loss=root_mean_squared_error)

model.fit(
    X_train_noisy, y_train_bin,
    epochs=1,
    batch_size=32,
    validation_data = (X_val_noisy, y_val_bin)
)
model.evaluate(X_test_noisy, y_test_bin)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 2)                 0         
_________________________________________________________________
dense_6 (Dense)              (None, 2)                 6         
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 3         
_________________________________________________________________
dense_8 (Dense)              (None, 2)                 4         
_________________________________________________________________
dense_9 (Dense)              (None, 2)                 6         
Total params: 19
Trainable params: 19
Non-trainable params: 0
_________________________________________________________________
Train on 1056863 samples, validate on 352288 samples
Epoch 1/5
1056863/1056863 [==============================] - 124s 117us/step - loss: 22299.63

nan

In [8]:
# normal autoencoder (sigmoid)

In [9]:
# joint (multi task learning model) loss

from numpy.random import seed
seed(0)
from tensorflow import set_random_seed
set_random_seed(0)

x_dim = X_train.shape[1]
x_input = Input(shape=(x_dim,))


hidden_enc = Dense(2, activation='sigmoid')(x_input)
# embedding is the user-item embedding (encoded representation)
embedding = Dense(embedding_dim, activation='sigmoid')(hidden_enc)
hidden_dec = Dense(2, activation='sigmoid')(embedding)
# reconstructed_input is the (decoded) reconstruction of the input 
# (lossy reconstruction)
reconstructed_input = Dense(x_dim, activation='sigmoid')(hidden_dec)
predictor = Dense(1, activation='sigmoid')(hidden_dec)



# Model(input, output): map the input to its reconstruction
# definition of autoencoder
joint_model = Model(inputs=x_input, outputs=[reconstructed_input, predictor])


joint_model.summary()

joint_model.compile(optimizer='rmsprop', loss=root_mean_squared_error)

joint_model.fit(
    x = [X_train_noisy], 
    y = [X_train, y_train_bin],
    epochs = 5,
    batch_size = 32,
    validation_data = ([X_val_noisy], [X_val, y_val_bin])
)
joint_model.evaluate([X_test_noisy], [X_test, y_test_bin])


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 2)            0                                            
__________________________________________________________________________________________________
dense_11 (Dense)                (None, 2)            6           input_3[0][0]                    
__________________________________________________________________________________________________
dense_12 (Dense)                (None, 1)            3           dense_11[0][0]                   
__________________________________________________________________________________________________
dense_13 (Dense)                (None, 2)            4           dense_12[0][0]                   
__________________________________________________________________________________________________
dense_14 (

[nan, nan, nan]

In [11]:
# 2 single (consecutive) loss

# separate
from numpy.random import seed
seed(0)
from tensorflow import set_random_seed
set_random_seed(0)


x_dim = X_train.shape[1]
x_input = Input(shape=(x_dim,))


hidden_enc = Dense(2, activation='sigmoid')(x_input)
# embedding is the user-item embedding (encoded representation)
embedding = Dense(embedding_dim, activation='sigmoid')(hidden_enc)
hidden_dec = Dense(2, activation='sigmoid')(embedding)
# reconstructed_input is the (decoded) reconstruction of the input 
# (lossy reconstruction)
reconstructed_input = Dense(x_dim, activation='sigmoid')(hidden_dec)


# Model(input, output): map the input to its reconstruction
# definition of autoencoder
autoencoder = Model(x_input, reconstructed_input)


# map the input to its embedding (encoded representation)
encoder = Model(x_input, embedding)


autoencoder.summary()

autoencoder.compile(optimizer='rmsprop', loss=root_mean_squared_error)

autoencoder.fit(
    X_train_noisy, X_train,
    epochs=5,
    batch_size=32,
    validation_data = (X_val_noisy, X_val)
)

# part 2
predictor = Dense(1, activation='sigmoid')(embedding)

model = Model(x_input, predictor)

model.summary()

model.compile(optimizer='rmsprop', loss=root_mean_squared_error)

model.fit(
    X_train_noisy, y_train_bin,
    epochs=1,
    batch_size=32,
    validation_data = (X_val_noisy, y_val_bin)
)
model.evaluate(X_test_noisy, y_test_bin)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, 2)                 0         
_________________________________________________________________
dense_21 (Dense)             (None, 2)                 6         
_________________________________________________________________
dense_22 (Dense)             (None, 1)                 3         
_________________________________________________________________
dense_23 (Dense)             (None, 2)                 4         
_________________________________________________________________
dense_24 (Dense)             (None, 2)                 6         
Total params: 19
Trainable params: 19
Non-trainable params: 0
_________________________________________________________________
Train on 1056863 samples, validate on 352288 samples
Epoch 1/5
1056863/1056863 [==============================] - 105s 99us/step - loss: 23135.381

nan

In [ ]:
# normal autoencoder (linear/identity)

In [13]:
# joint (multi task learning model) loss

from numpy.random import seed
seed(0)
from tensorflow import set_random_seed
set_random_seed(0)

x_dim = X_train.shape[1]
x_input = Input(shape=(x_dim,))


hidden_enc = Dense(2, activation='linear')(x_input)
# embedding is the user-item embedding (encoded representation)
embedding = Dense(embedding_dim, activation='linear')(hidden_enc)
hidden_dec = Dense(2, activation='linear')(embedding)
# reconstructed_input is the (decoded) reconstruction of the input 
# (lossy reconstruction)
reconstructed_input = Dense(x_dim, activation='linear')(hidden_dec)
predictor = Dense(1, activation='linear')(hidden_dec)



# Model(input, output): map the input to its reconstruction
# definition of autoencoder
joint_model = Model(inputs=x_input, outputs=[reconstructed_input, predictor])


joint_model.summary()

joint_model.compile(optimizer='rmsprop', loss=root_mean_squared_error)

joint_model.fit(
    x = [X_train_noisy], 
    y = [X_train, y_train_bin],
    epochs = 5,
    batch_size = 32,
    validation_data = ([X_val_noisy], [X_val, y_val_bin])
)
joint_model.evaluate([X_test_noisy], [X_test, y_test_bin])

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            (None, 2)            0                                            
__________________________________________________________________________________________________
dense_31 (Dense)                (None, 2)            6           input_7[0][0]                    
__________________________________________________________________________________________________
dense_32 (Dense)                (None, 1)            3           dense_31[0][0]                   
__________________________________________________________________________________________________
dense_33 (Dense)                (None, 2)            4           dense_32[0][0]                   
__________________________________________________________________________________________________
dense_34 (

[59.405838737934786, 38.30236802390036, 21.103470714987324]

In [5]:
# 2 single (consecutive) loss

# separate
from numpy.random import seed
seed(0)
from tensorflow import set_random_seed
set_random_seed(0)


x_dim = X_train.shape[1]
x_input = Input(shape=(x_dim,))


hidden_enc = Dense(2, activation='linear')(x_input)
# embedding is the user-item embedding (encoded representation)
embedding = Dense(embedding_dim, activation='linear')(hidden_enc)
hidden_dec = Dense(2, activation='linear')(embedding)
# reconstructed_input is the (decoded) reconstruction of the input 
# (lossy reconstruction)
reconstructed_input = Dense(x_dim, activation='linear')(hidden_dec)


# Model(input, output): map the input to its reconstruction
# definition of autoencoder
autoencoder = Model(x_input, reconstructed_input)


# map the input to its embedding (encoded representation)
encoder = Model(x_input, embedding)


autoencoder.summary()

autoencoder.compile(optimizer='rmsprop', loss=root_mean_squared_error)

autoencoder.fit(
    X_train_noisy, X_train,
    epochs=3,
    batch_size=32,
    validation_data = (X_val_noisy, X_val)
)

# part 2
predictor = Dense(1, activation='linear')(embedding)

model = Model(x_input, predictor)

model.summary()

model.compile(optimizer='rmsprop', loss=root_mean_squared_error)

model.fit(
    X_train_noisy, y_train_bin,
    epochs=1,
    batch_size=32,
    validation_data = (X_val_noisy, y_val_bin)
)
model.evaluate(X_test_noisy, y_test_bin)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 2)                 0         
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 6         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 3         
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 4         
_________________________________________________________________
dense_4 (Dense)              (None, 2)                 6         
Total params: 19
Trainable params: 19
Non-trainable params: 0
_________________________________________________________________
Train on 1056863 samples, validate on 352288 samples
Epoch 1/3
1056863/1056863 [==============================] - 42s 40us/step - loss: 208.6167 -

0.34491201390272286

# continuous label (-10, 10)

In [15]:
# load labels
y_train_cat =  np.load("J2_ytrCat.npy")
y_val_cat =  np.load("J2_yvalCat.npy")
y_test_cat =  np.load("J2_ytestCat.npy")

In [16]:
# normal autoencoder (Selu)

In [17]:
# joint (multi task learning model) loss

from numpy.random import seed
seed(0)
from tensorflow import set_random_seed
set_random_seed(0)

x_dim = X_train.shape[1]
x_input = Input(shape=(x_dim,))


hidden_enc = Dense(2, activation='selu')(x_input)
# embedding is the user-item embedding (encoded representation)
embedding = Dense(embedding_dim, activation='selu')(hidden_enc)
hidden_dec = Dense(2, activation='selu')(embedding)
# reconstructed_input is the (decoded) reconstruction of the input 
# (lossy reconstruction)
reconstructed_input = Dense(x_dim, activation='selu')(hidden_dec)
predictor = Dense(1, activation='selu')(hidden_dec)



# Model(input, output): map the input to its reconstruction
# definition of autoencoder
joint_model = Model(inputs=x_input, outputs=[reconstructed_input, predictor])


joint_model.summary()

joint_model.compile(optimizer='rmsprop', loss=root_mean_squared_error)

joint_model.fit(
    x = [X_train_noisy], 
    y = [X_train, y_train_cat],
    epochs = 5,
    batch_size = 32,
    validation_data = ([X_val_noisy], [X_val, y_val_cat])
)
joint_model.evaluate([X_test_noisy], [X_test, y_test_cat])

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_9 (InputLayer)            (None, 2)            0                                            
__________________________________________________________________________________________________
dense_41 (Dense)                (None, 2)            6           input_9[0][0]                    
__________________________________________________________________________________________________
dense_42 (Dense)                (None, 1)            3           dense_41[0][0]                   
__________________________________________________________________________________________________
dense_43 (Dense)                (None, 2)            4           dense_42[0][0]                   
__________________________________________________________________________________________________
dense_44 (

[1999.6215000559735, 1995.2686647880694, 4.352835025998723]

In [18]:
# 2 single (consecutive) loss

# separate
from numpy.random import seed
seed(0)
from tensorflow import set_random_seed
set_random_seed(0)



x_dim = X_train.shape[1]
x_input = Input(shape=(x_dim,))


hidden_enc = Dense(2, activation='selu')(x_input)
# embedding is the user-item embedding (encoded representation)
embedding = Dense(embedding_dim, activation='selu')(hidden_enc)
hidden_dec = Dense(2, activation='selu')(embedding)
# reconstructed_input is the (decoded) reconstruction of the input 
# (lossy reconstruction)
reconstructed_input = Dense(x_dim, activation='selu')(hidden_dec)


# Model(input, output): map the input to its reconstruction
# definition of autoencoder
autoencoder = Model(x_input, reconstructed_input)


# map the input to its embedding (encoded representation)
encoder = Model(x_input, embedding)


autoencoder.summary()

autoencoder.compile(optimizer='rmsprop', loss=root_mean_squared_error)

autoencoder.fit(
    X_train_noisy, X_train,
    epochs=5,
    batch_size=32,
    validation_data = (X_val_noisy, X_val)
)

# part 2
predictor = Dense(1, activation='selu')(embedding)

model = Model(x_input, predictor)

model.summary()

model.compile(optimizer='rmsprop', loss=root_mean_squared_error)

model.fit(
    X_train_noisy, y_train_cat,
    epochs=1,
    batch_size=32,
    validation_data = (X_val_noisy, y_val_cat)
)
model.evaluate(X_test_noisy, y_test_cat)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_10 (InputLayer)        (None, 2)                 0         
_________________________________________________________________
dense_46 (Dense)             (None, 2)                 6         
_________________________________________________________________
dense_47 (Dense)             (None, 1)                 3         
_________________________________________________________________
dense_48 (Dense)             (None, 2)                 4         
_________________________________________________________________
dense_49 (Dense)             (None, 2)                 6         
Total params: 19
Trainable params: 19
Non-trainable params: 0
_________________________________________________________________
Train on 1056863 samples, validate on 352288 samples
Epoch 1/5
1056863/1056863 [==============================] - 108s 103us/step - loss: 22299.65

4.3168460608546555

In [19]:
# joint (multi task learning model) loss

from numpy.random import seed
seed(0)
from tensorflow import set_random_seed
set_random_seed(0)

x_dim = X_train.shape[1]
x_input = Input(shape=(x_dim,))


hidden_enc = Dense(2, activation='sigmoid')(x_input)
# embedding is the user-item embedding (encoded representation)
embedding = Dense(embedding_dim, activation='sigmoid')(hidden_enc)
hidden_dec = Dense(2, activation='sigmoid')(embedding)
# reconstructed_input is the (decoded) reconstruction of the input 
# (lossy reconstruction)
reconstructed_input = Dense(x_dim, activation='sigmoid')(hidden_dec)
predictor = Dense(1, activation='sigmoid')(hidden_dec)



# Model(input, output): map the input to its reconstruction
# definition of autoencoder
joint_model = Model(inputs=x_input, outputs=[reconstructed_input, predictor])


joint_model.summary()

joint_model.compile(optimizer='rmsprop', loss=root_mean_squared_error)

joint_model.fit(
    x = [X_train_noisy], 
    y = [X_train, y_train_cat],
    epochs = 5,
    batch_size = 32,
    validation_data = ([X_val_noisy], [X_val, y_val_cat])
)
joint_model.evaluate([X_test_noisy], [X_test, y_test_cat])


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_11 (InputLayer)           (None, 2)            0                                            
__________________________________________________________________________________________________
dense_51 (Dense)                (None, 2)            6           input_11[0][0]                   
__________________________________________________________________________________________________
dense_52 (Dense)                (None, 1)            3           dense_51[0][0]                   
__________________________________________________________________________________________________
dense_53 (Dense)                (None, 2)            4           dense_52[0][0]                   
__________________________________________________________________________________________________
dense_54 (

[nan, nan, nan]

In [20]:
# 2 single (consecutive) loss

# separate
from numpy.random import seed
seed(0)
from tensorflow import set_random_seed
set_random_seed(0)


x_dim = X_train.shape[1]
x_input = Input(shape=(x_dim,))


hidden_enc = Dense(2, activation='sigmoid')(x_input)
# embedding is the user-item embedding (encoded representation)
embedding = Dense(embedding_dim, activation='sigmoid')(hidden_enc)
hidden_dec = Dense(2, activation='sigmoid')(embedding)
# reconstructed_input is the (decoded) reconstruction of the input 
# (lossy reconstruction)
reconstructed_input = Dense(x_dim, activation='sigmoid')(hidden_dec)


# Model(input, output): map the input to its reconstruction
# definition of autoencoder
autoencoder = Model(x_input, reconstructed_input)


# map the input to its embedding (encoded representation)
encoder = Model(x_input, embedding)


autoencoder.summary()

autoencoder.compile(optimizer='rmsprop', loss=root_mean_squared_error)

autoencoder.fit(
    X_train_noisy, X_train,
    epochs=5,
    batch_size=32,
    validation_data = (X_val_noisy, X_val)
)

# part 2
predictor = Dense(1, activation='sigmoid')(embedding)

model = Model(x_input, predictor)

model.summary()

model.compile(optimizer='rmsprop', loss=root_mean_squared_error)

model.fit(
    X_train_noisy, y_train_cat,
    epochs=1,
    batch_size=32,
    validation_data = (X_val_noisy, y_val_cat)
)
model.evaluate(X_test_noisy, y_test_cat)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_12 (InputLayer)        (None, 2)                 0         
_________________________________________________________________
dense_56 (Dense)             (None, 2)                 6         
_________________________________________________________________
dense_57 (Dense)             (None, 1)                 3         
_________________________________________________________________
dense_58 (Dense)             (None, 2)                 4         
_________________________________________________________________
dense_59 (Dense)             (None, 2)                 6         
Total params: 19
Trainable params: 19
Non-trainable params: 0
_________________________________________________________________
Train on 1056863 samples, validate on 352288 samples
Epoch 1/5
1056863/1056863 [==============================] - 116s 110us/step - loss: 23135.38

nan

In [21]:
# normal autoencoder (linear/identity)

In [22]:
# joint (multi task learning model) loss

from numpy.random import seed
seed(0)
from tensorflow import set_random_seed
set_random_seed(0)

x_dim = X_train.shape[1]
x_input = Input(shape=(x_dim,))


hidden_enc = Dense(2, activation='linear')(x_input)
# embedding is the user-item embedding (encoded representation)
embedding = Dense(embedding_dim, activation='linear')(hidden_enc)
hidden_dec = Dense(2, activation='linear')(embedding)
# reconstructed_input is the (decoded) reconstruction of the input 
# (lossy reconstruction)
reconstructed_input = Dense(x_dim, activation='linear')(hidden_dec)
predictor = Dense(1, activation='linear')(hidden_dec)



# Model(input, output): map the input to its reconstruction
# definition of autoencoder
joint_model = Model(inputs=x_input, outputs=[reconstructed_input, predictor])


joint_model.summary()

joint_model.compile(optimizer='rmsprop', loss=root_mean_squared_error)

joint_model.fit(
    x = [X_train_noisy], 
    y = [X_train, y_train_cat],
    epochs = 5,
    batch_size = 32,
    validation_data = ([X_val_noisy], [X_val, y_val_cat])
)


joint_model.evaluate([X_test_noisy], [X_test, y_test_cat])

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_13 (InputLayer)           (None, 2)            0                                            
__________________________________________________________________________________________________
dense_61 (Dense)                (None, 2)            6           input_13[0][0]                   
__________________________________________________________________________________________________
dense_62 (Dense)                (None, 1)            3           dense_61[0][0]                   
__________________________________________________________________________________________________
dense_63 (Dense)                (None, 2)            4           dense_62[0][0]                   
__________________________________________________________________________________________________
dense_64 (

[56.86095061273165, 46.625389081102725, 10.235561532928331]

In [24]:
# 2 single (consecutive) loss

# separate
from numpy.random import seed
seed(0)
from tensorflow import set_random_seed
set_random_seed(0)


x_dim = X_train.shape[1]
x_input = Input(shape=(x_dim,))


hidden_enc = Dense(2, activation='linear')(x_input)
# embedding is the user-item embedding (encoded representation)
embedding = Dense(embedding_dim, activation='linear')(hidden_enc)
hidden_dec = Dense(2, activation='linear')(embedding)
# reconstructed_input is the (decoded) reconstruction of the input 
# (lossy reconstruction)
reconstructed_input = Dense(x_dim, activation='linear')(hidden_dec)


# Model(input, output): map the input to its reconstruction
# definition of autoencoder
autoencoder = Model(x_input, reconstructed_input)


# map the input to its embedding (encoded representation)
encoder = Model(x_input, embedding)


autoencoder.summary()

autoencoder.compile(optimizer='rmsprop', loss=root_mean_squared_error)

autoencoder.fit(
    X_train_noisy, X_train,
    epochs=3,
    batch_size=32,
    validation_data = (X_val_noisy, X_val)
)

# part 2
predictor = Dense(1, activation='linear')(embedding)

model = Model(x_input, predictor)

model.summary()

model.compile(optimizer='rmsprop', loss=root_mean_squared_error)

model.fit(
    X_train_noisy, y_train_cat,
    epochs=1,
    batch_size=32,
    validation_data = (X_val_noisy, y_val_cat)
)
model.evaluate(X_test_noisy, y_test_cat)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_15 (InputLayer)        (None, 2)                 0         
_________________________________________________________________
dense_71 (Dense)             (None, 2)                 6         
_________________________________________________________________
dense_72 (Dense)             (None, 1)                 3         
_________________________________________________________________
dense_73 (Dense)             (None, 2)                 4         
_________________________________________________________________
dense_74 (Dense)             (None, 2)                 6         
Total params: 19
Trainable params: 19
Non-trainable params: 0
_________________________________________________________________
Train on 1056863 samples, validate on 352288 samples
Epoch 1/3
1056863/1056863 [==============================] - 69s 65us/step - loss: 208.9652 -

4.268334292598832